In [26]:
import sys
sys.dont_write_bytecode = True
import json

from utils import *
from gen_config_arr import *
from gen_bins import *

def replace_path(aec):
    paths = aec["paths"]
    for key in paths:
       for app in aec["apps"]:
           flags = aec["apps"][app].keys()
           for f in flags:
               fv = aec["apps"][app][f]
               if type(fv) == str:
                   aec["apps"][app][f] = fv.replace(f"$({key})", paths[key])
               elif type(fv) == list:
                   for i in range(len(fv)):
                       fv[i] = fv[i].replace(f"$({key})", paths[key])

sc = load_config("../../../config.json") ## system config
aec = load_config("../configs/default_expv2.json") ## apps evaluation config

board_user = sc["board_user"]
data_dir_host = sc["data_dir"]
board_eval_dir = sc["board_eval_dir"]



replace_path(aec)


In [30]:
import json
from itertools import product
from gen_bins import gen_bins

cpu_paths = {
    "benchmark_model": ["tensorflow/lite/tools/benchmark", "benchmark_model"],
    "inference_diff": [
        "tensorflow/lite/tools/evaluation/tasks/inference_diff",
        "run_eval",
    ],
    "eval_model": ["tensorflow/lite/examples/secda_apps/eval_model", "eval_model"],
    "eval_model_accuracy": ["tensorflow/lite/examples/secda_apps/eval_model_accuracy", "eval_model_accuracy"],
    "imagenet_image_classification": ["tensorflow/lite/examples/secda_apps/imagenet_image_classification", "run_eval"],
}

def get_models_from_dir(model_dir):
    models = []
    for model in os.listdir(model_dir):
        if model.endswith(".tflite"):
            models.append(model_dir + model.replace(".tflite", ""))
    return models


def get_hw_info(
    sc,
    hardware,
):
    ## Generate configs.sh
    config_list = []
    hw_list = []
    version_list = []
    del_version_list = []
    delegate_list = []
    for hw in hardware:
        hw_config_file= find_hw_config(f"{sc['secda_tflite_path']}/{sc['hw_configs']}", hw)
        hw_config = load_config(hw_config_file)
        hw_list.append(hw_config["acc_name"])
        version_list.append(hw_config["version"])
        del_version_list.append(hw_config["del_version"])
        delegate_list.append(hw_config["del"])
        config_list.append(hw_config)



def create_run_config(sc,aec ,app_dict):
    hw_list = []
    app_list = []
    cmd_list = []
    del_version_list = []
    delegate_list = []
    taglist = []
    out_dir = f"./{sc['out_dir']}"
    for hw in aec["hardware"]:
        hw_config_file= find_hw_config(f"{sc['secda_tflite_path']}/{sc['hw_configs']}", hw)
        hw_config = load_config(hw_config_file)
        delegate = hw_config["del"]
        for app in aec["apps"]:
            print(f"Generating config array for {app} on {hw}")
            flags = aec["apps"][app].keys()
            config_arr = {}

            for f in flags:
                fv = aec["apps"][app][f]
                if f == "models" and type(fv) == str and fv.endswith("/"):
                    models = get_models_from_dir(aec["apps"][app][f])
                    # add data_dir_host to models
                    models = [model + ".tflite" for model in models]
                    print(models)
                    config_arr[f] = models
                    continue
                elif f == "models" and type(fv) == list:
                    models = []
                    for model in fv:
                        if model.endswith("/"):
                            models += get_models_from_dir(model)
                        else:
                            models.append(model)
                    # add data_dir_host to models
                    models = [model + ".tflite" for model in models]
                    print(models)
                    config_arr[f] = models
                    continue

                # check if flag is a list
                if type(aec["apps"][app][f]) == list:
                    config_arr[f] = aec["apps"][app][f]
                else:
                    config_arr[f] = [aec["apps"][app][f]]
            all_configs = list(product(*config_arr.values()))
            print(f"Total number of configurations: {len(all_configs)}")
            # print(all_configs)
            usedel=f"--use_{delegate}=true"
            if hw == "CPU":
                usedel=""
            for config in all_configs:
                flags_str = ""
                tag = f"{app}_{hw_config['acc_name']}_{hw_config['del']}_{hw_config['del_version']}"
                for i, f in enumerate(config_arr.keys()):
                    flags_str += f" --{f} {config[i]}" 
                    if config[i].endswith(".tflite"):
                        s = config[i].split("/")[-1]
                        tag += f"_{s}"
                    else:
                        tag += f"_{config[i]}"
                app_call = f"{board_eval_dir}/bin/{app_dict[app]}_{hw_config['del']}_{hw_config['del_version']}"
                cmd = f"{app_call} {flags_str} {usedel}"
                hw_list.append(hw_config["acc_name"])
                del_version_list.append(hw_config["del_version"])
                delegate_list.append(hw_config["del"])
                app_list.append(app)
                cmd_list.append(cmd)
                taglist.append(tag)

        os.makedirs(out_dir, exist_ok=True)
        f = open(f"{out_dir}/configs_bm.sh", "w+")
        # list of all the config properties
        declare_array(f, "hw", hw_list)
        declare_array(f, "del_version", del_version_list)
        declare_array(f, "del", delegate_list)
        declare_array(f, "app_list", app_list)
        declare_array(f, "cmd_list", cmd_list)
        declare_array(f, "taglist", taglist)
        f.close()

def create_bin_config(sc,aec):
    apps = aec["apps"].keys()
    hw = aec["hardware"]
    app_dict = {}
    for app in apps:
        sn = ""
        for s in app.split("_"):
            sn += s[0]
        app_dict[app] = sn
    
    gen_bins(sc, hw, app_dict)
    return app_dict

applist = create_bin_config(sc,aec)
create_run_config(sc,aec,applist)
                    

Generating config array for eval_model on CPU
['/home/jude/Workspace/SECDA-TFLite_v1.2/data/imagenet/models/mobilenetv2.tflite', '/home/jude/Workspace/SECDA-TFLite_v1.2/data/imagenet/models/resnet18v1.tflite', '/home/jude/Workspace/SECDA-TFLite_v1.2/data/imagenet/models/resnet50v2.tflite']
Total number of configurations: 6
Generating config array for eval_model_accuracy on CPU
['/home/jude/Workspace/SECDA-TFLite_v1.2/data/cifar10/models/mbV1DownloadedTrained_Cifer10.tflite', '/home/jude/Workspace/SECDA-TFLite_v1.2/data/cifar10/models/mobilentv2_fpTrained_ptqint8_cifar10_100.tflite']
Total number of configurations: 4
Generating config array for imagenet_image_classification on CPU
['/home/jude/Workspace/SECDA-TFLite_v1.2/data/imagenet/models/mobilenetv2.tflite', '/home/jude/Workspace/SECDA-TFLite_v1.2/data/imagenet/models/resnet18v1.tflite', '/home/jude/Workspace/SECDA-TFLite_v1.2/data/imagenet/models/resnet50v2.tflite']
Total number of configurations: 6
Generating config array for benc

'/home/xilinx/Workspace/secda_benchmark_suite/bins/id_${DEL}_${DEL_VERSION}'